# Step 2- Cleaning and Merging Data
## Cleaning before merging:
During data cleaning, I performed the following steps:

Loaded all datasets and ensured key columns (IDs) had consistent data types- string.
Converted timestamps to proper datetime format.
Checked for and removed duplicate records.
Identified missing values and dropped rows missing critical identifiers while keeping non-critical missing fields.
Standardized categorical fields such as transaction type for consistency.
Mapped the MCC code descriptions using the JSON file.
Converting data type to a standardised format
Converted all ID columns (transaction_id, card_id, user_id, mcc_code) to strings because IDs are identifiers, not numerical values. This is an important step to ensure consistent merging

In [ ]:
# Convert IDs to string (important for merging)
transactions["card_id"] = transactions["card_id"].astype(str)
cards["card_id"] = cards["card_id"].astype(str)

cards["user_id"] = cards["user_id"].astype(str)
transactions["user_id"] = transactions["user_id"].astype(str)
users["user_id"] = users["user_id"].astype(str)

transactions["transaction_id"] = transactions["transaction_id"].astype(str)
train_fraud["transaction_id"] = train_fraud["transaction_id"].astype(str)

# For MCC codes
transactions["mcc_code"] = transactions["mcc_code"].astype(str)
mcc_df["mcc_code"] = mcc_df["mcc_code"].astype(str)

Converting date columns to datetime format
This step is important because it will allow us:

To sort transactions by time
To do the extraction of year/month/day/hour
To do time-series analysis

In [ ]:
transactions['date'] = pd.to_datetime(
    transactions['date'],
    format='%d/%m/%Y %H:%M'
)

transactions['date'].head

In [ ]:
cards['expiry_date'] = pd.to_datetime(cards['expiry_date'], format='%b-%y')
cards['acct_open_date'] = pd.to_datetime(cards['acct_open_date'], format='%b-%y')

cards['expiry_date'].head()
cards['acct_open_date'].head()

In [ ]:
# remove $ and commas
transactions['amount'] = transactions['amount'].replace('[\$,]', '', regex=True)

# Convert amount to float- 
transactions['amount'] = pd.to_numeric(transactions['amount'])

# Verify conversion
transactions['amount'].dtype   # should be float64
transactions['amount'].head()

In [ ]:
# Converting cedit_limit to numeric datatype
cards['credit_limit'] = cards['credit_limit'].replace('[\$,]', '', regex=True)
cards['credit_limit'] = pd.to_numeric(cards['credit_limit'])

# Converting per_capita_income to numeric datatype
users['per_capita_income'] = users['per_capita_income'].replace('[\$,]', '', regex=True)
users['per_capita_income'] = pd.to_numeric(users['per_capita_income'])

# Converting yearly_income to numeric datatype
users['yearly_income'] = users['yearly_income'].replace('[\$,]', '', regex=True)
users['yearly_income'] = pd.to_numeric(users['yearly_income'])

# Converting total_debt to numeric datatype
users['total_debt'] = users['total_debt'].replace('[\$,]', '', regex=True)
users['total_debt'] = pd.to_numeric(users['total_debt'])

cards['credit_limit'].head()
users['per_capita_income'].head()
users['yearly_income'].head()
users['total_debt'].head()

# Handling Missing Values
Checking missing values in each table

In [ ]:
# Checking missing values in all the tables
transactions.isna().sum()
cards.isna().sum()
users.isna().sum()
mcc_df.isna().sum()
train_fraud.isna().sum()


The above code identified some missing values in the transactions table in merchant_state, zip code, and errors columns, which is not an alarming situation as it is completely normal for a merchant not to provide state or zip code information, especially during online or international transactions. In the 'errors' column, a missing value shows that there was 'no error' during the transaction which is good.

As none of the missing values occur in critical join keys, there is no need to drop any row. We will fill the missing values with an appropriate value.



In [ ]:
# Handling the missing values in merchant_state, merchant_city and errors 

transactions['merchant_state'] = transactions['merchant_state'].fillna("Unknown")

# convert to string since zip is float here
transactions['zip'] = transactions['zip'].fillna("00000").astype(str)  
transactions['errors'] = transactions['errors'].fillna("No Error")

# Verify if missing values have gone in the transactions table

transactions[['merchant_state','zip','errors']].isna().sum()


# Removing Duplicate Records

In [ ]:
# Checking for duplicates
duplicates = transactions.duplicated().sum()
duplicates = cards.duplicated().sum()
duplicates = users.duplicated().sum()
duplicates = train_fraud.duplicated().sum()
duplicates = mcc_df.duplicated().sum()

In [ ]:
# Remove if found any
transactions.drop_duplicates(inplace=True)
cards.drop_duplicates(inplace=True)
users.drop_duplicates(inplace=True)
mcc_df.drop_duplicates(inplace=True)
train_fraud.drop_duplicates(inplace=True)

# Merging the data in all the tables into a Single DataFrame
## Merged all tables into one final dataframe using card_id, user_id, transaction_id and mcc_code

In [ ]:
# Merge transactions + cards

merged_df = transactions.merge(
    cards,
    on="card_id",
    how="left"
)


# Merge with user data

merged_df = merged_df.merge(
    users,
    on="user_id",
    how="left"
)


# Merge with MCC codes

merged_df = merged_df.merge(
    mcc_df,
    on="mcc_code",
    how="left"
)


# Merge with train fraud labels

merged_df = merged_df.merge(
    train_fraud,
    on="transaction_id",
    how="left"
)


# Final check

print("Final dataframe shape:", merged_df.shape)
merged_df.head()

# Secondary Cleaning- Clean after merging
### Sometimes the final dataframe which is generated by joining multiple tables needs cleaning-

## 1. Handle missing values introduced by merge

In [ ]:
# Check how many missing values are in the new dataframe
merged_df.isna().sum()

In [ ]:
# Fill missing merchant_state ,zip and error
merged_df['merchant_state'] = merged_df['merchant_state'].fillna('Unknown')
merged_df['zip'] = merged_df['zip'].fillna('00000')

# For errors column
merged_df['errors'] = merged_df['errors'].fillna('No Error')

# Optional: Check how many missing remain
merged_df.isna().sum()